## Why is Community Engagement Rank a good choice for Farcaster?

Community Engagement Rank is an excellent choice for analyzing the Farcaster ecosystem for several reasons:

**1. Decentralized Social Network**: Farcaster is a decentralized social protocol, making engagement metrics crucial for understanding user interactions and influence.

**2. User Influence**: It helps identify the most influential and active users within the Farcaster community.

**3. Content Quality**: By analyzing reactions and mentions, we can gauge the quality and impact of content shared on the platform.

**4. Network Growth**: It provides insights into how the Farcaster network is growing and evolving over time.



In [16]:
#Install necessary package
!pip install openrank_sdk 

## OpenRank: A Brief Overview

OpenRank is an open-source implementation of the EigenTrust algorithm, designed to compute trust scores in decentralized networks. It's particularly useful for analyzing and ranking entities in Web3 ecosystems, such as blockchain networks, decentralized social media platforms, and other peer-to-peer systems.

Key features of OpenRank include:

**1. Decentralized Trust Computation**: It calculates trust scores without relying on a central authority, making it ideal for decentralized networks.

**2. Customizable**: It allows for the incorporation of pre-existing trust information (pretrust) and can be adapted to various types of interaction data.

**3. Scalability**: OpenRank is designed to handle large-scale networks with millions of nodes and interactions.

OpenRank is a powerful algorithm that operates on reputation graph data, which is constructed from peer-to-peer trust signals within a network. At its core, OpenRank interprets and analyzes complex webs of trust relationships, answering the fundamental question: "Who trusts whom, by how much, and for what purpose?"

In this context:

    1. Who (The Truster): Represents the entity extending trust. This could be a user, a node, or any participant in the network capable of expressing trust.

    2. Whom (The Trustee): Denotes the recipient of trust. This is the entity being evaluated or trusted within the network.

    3. How Much: Quantifies the degree or intensity of trust from the truster to the trustee. This could be represented as a numerical value or weight.

    4. What (The Context): Specifies the nature or type of trust being extended. This crucial element provides context, defining the role or specific domain in which the trust is relevant.


In the context of our Community Engagement Rank for Farcaster, OpenRank will use the interaction data (reactions and mentions) to compute trust scores for users. These scores will reflect each user's influence and engagement level within the Farcaster community, based on how often they're interacted with and mentioned by others.

The OpenRank algorithm takes two main inputs:

    1. Localtrust: This is derived from the direct interactions between users, which in our case comes from the SQL query results showing reactions and mentions.

    2. Pretrust: This is a set of nodes (users) that are considered trustworthy from the outset. For Farcaster, this could be based on verified users or other criteria. We will be using Power Badge users from warpcast platform.

By applying OpenRank to our Farcaster data, we can obtain a nuanced and sybil-resistant ranking of users based on their community engagement, providing valuable insights into the dynamics of the Farcaster ecosystem.

## Power Badges

Power badges are awarded to active, engaging, and non-spammy users on Warpcast. Benefits include:

    1. Profile badge
    2. Higher search ranking
    3. Access to restricted channels
    4. Increased follow recommendations

Developers can access the list of badge holders via API for various purposes.

The selection process is based on a PageRank-like system considering:

    1. Recent activity
    2. Engagement from other power users
    3. Absence of spam flags

To increase chances of earning a power badge:

    1. Use Farcaster regularly
    2. Create content that appeals to power users
    3. Avoid posting spam or off-topic content

This system aims to recognize and reward valuable contributors to the Warpcast community.

## LocalTrust

Our Local Trust query for Farcaster focuses on two main types of interactions:

    1. Reactions: When a user reacts to another user's cast (post).
    2. Mentions: When a user mentions another user in their cast.

The query aggregates these interactions over the past 30 days to create a recent engagement score. We use the OpenRank algorithm to calculate the final rank based on these interactions.

In [ ]:
## we will be using flipsidecrypto to get the data and export it as "LocalTrust.csv"

WITH reactions AS (
    SELECT 
        FID
        TO_VARCHAR(TARGET_HASH) AS interaction_id,
        COUNT(*) AS interactions
    FROM 
        external.farcaster.fact_reactions
    WHERE 
        TIMESTAMP >= DATEADD(day, -30, CURRENT_DATE())
        AND DELETED_AT IS NULL
    GROUP BY 
        FID, TARGET_HASH
),
mentions AS (
    SELECT 
        FID
        m.value AS interaction_id,
        COUNT(*) AS interactions
    FROM 
        external.farcaster.fact_casts,
        LATERAL FLATTEN(PARSE_JSON(MENTIONS)) m
    WHERE 
        TIMESTAMP >= DATEADD(day, -30, CURRENT_DATE())
        AND DELETED_AT IS NULL
    GROUP BY 
        FID, m.value
)
SELECT fid, interaction_id, interactions
FROM reactions
UNION ALL
SELECT fid, interaction_id, interactions
FROM mentions ORDER BY interactions DESC
Limit 100000

In [1]:
## let read the data
import pandas as pd

localtrust = pd.read_csv("LocalTrust.csv")
localtrust.head()

,FID,INTERACTION_ID,INTERACTIONS
0,403507,781098,87232.0
1,538721,503116,10512.0
2,284439,284439,6809.0
3,284439,8446,5764.0
4,405154,244062,5227.0


In [2]:
localtrust = localtrust[localtrust['FID'] != localtrust['INTERACTION_ID']]
localtrust = localtrust[localtrust['INTERACTIONS']>10]

In [3]:
localtrust.shape

(25103, 3)

## PreTrust

As said earlier, we will be use power badge users.

In [3]:
#we will user warpcast api to get the current power badge users.
!curl https://api.warpcast.com/v2/power-badge-users | jq > power-badge.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11937  100 11937    0     0  11282      0  0:00:01  0:00:01 --:--:-- 11282


In [4]:
import json

with open("power-badge.json", "r") as f:
    powerBadgeUsers = json.load(f)

In [5]:
pretrust = [str(x) for x in powerBadgeUsers['result']['fids']]

## OpenRank

In [8]:
#let's select only those power badge user who are in localtrust
all_fids = set(localtrust['FID']).union(set(localtrust['INTERACTION_ID']))
pretrust = [x for x in pretrust if x in localtrust['FID']]

In [9]:
# create the graphs from LocalTrust and preTrust

localtrust_graph = [{'i': row['FID'], 'j':row['INTERACTION_ID'], 'v':row['INTERACTIONS']} for i, row in localtrust.iterrows()]
pretrust_graph = [{'i':fid, 'v':10000} for fid in pretrust]

In [10]:
# Use openrank
from openrank_sdk import EigenTrust

a = EigenTrust()
farcaster_user_rank = a.run_eigentrust(localtrust_graph, pretrust_graph)
farcaster_user_rank[:10]

[{'i': '537227', 'v': 0.03328530686863001},
 {'i': '321969', 'v': 0.02127077065196924},
 {'i': '12741', 'v': 0.019893169985400347},
 {'i': '781098', 'v': 0.012732215949724532},
 {'i': '343261', 'v': 0.010746917761456441},
 {'i': '366713', 'v': 0.01012369557665069},
 {'i': '388302', 'v': 0.008823624232050832},
 {'i': '531162', 'v': 0.008436076346563993},
 {'i': '602', 'v': 0.00703637422581674},
 {'i': '20909', 'v': 0.006207504371209033}]